In [ ]:
import os
import warnings
import pandas as pd
import seaborn as sns

warnings.filterwarnings(action='ignore', message='numpy.dtype size changed')
warnings.filterwarnings(action='ignore', message='compiletime version 3.5 of module')

if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
os.chdir(os.path.split(workbookDir)[0])

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from synthesized.testing.evaluation import Evaluation
evaluation = Evaluation(config_path='configs/evaluation/synthetic_distributions.json', name='synthetic') # use this line if you want to run this notbook manually
# evaluation = Evaluation()

In [ ]:
from synthesized import BasicSynthesizer

In [ ]:
import numpy as np

In [ ]:
def create_ball(x_mean, x_std, y_mean, y_std, size, cor=0.):
    mean = [x_mean, y_mean]
    cov = [[x_std**2, x_std*y_std*cor], [x_std*y_std*cor, y_std**2]]
    x, y = np.random.multivariate_normal(mean, cov, size).T
    df = pd.DataFrame({'x': x, 'y': y})
    return df

In [ ]:
def create_line(x_range, intercept, slope, y_std, size):
    x = np.random.uniform(low=x_range[0], high=x_range[1], size=size)
    y = intercept + x*slope + np.random.normal(loc=0, scale=y_std, size=size)
    df = pd.DataFrame({'x': x, 'y': y})
    return df

In [ ]:
from scipy.stats import powerlaw
def create_power_law_distribution(a, scale, size):
    return pd.DataFrame({'x': scale*powerlaw.rvs(a, size=size)})

In [ ]:
from scipy.stats import bernoulli
def create_bernoulli_distribution(ratio, size):
    df = pd.DataFrame({'x': bernoulli.rvs(ratio, size=size)})
    df['x'] = df['x'].astype('category')
    return df

In [ ]:
def create_conditional_distibution(*norm_params, size):
    df = pd.DataFrame()
    for i, (mean, std) in enumerate(norm_params):
        x = [i] * size
        y = np.random.normal(mean, std, size)
        df = df.append(pd.DataFrame({'x': x, 'y': y}), ignore_index=True)
    df = df.sample(frac=1).reset_index(drop=True)
    df['x'] = df['x'].astype('category')
    return df

In [ ]:
def create_unifom_categorical(n_classes, size):
    df = pd.DataFrame({'x': range(n_classes)})
    df = df.sample(10000, replace=True)
    df['x'] = df['x'].astype('category')
    return df

In [ ]:
def create_power_law_categorical(n_classes, size):
    sample = [j for i in range(n_classes) for j in [i]*2**(n_classes-i-1)]
    df = pd.DataFrame({'x': sample})
    df = df.sample(10000, replace=True)
    df['x'] = df['x'].astype('category')
    return df

In [ ]:
def plot_data(data, ax=None):
    if data.shape[1] == 1:
        if data['x'].dtype.kind == 'O':
            return sns.distplot(data, ax=ax, kde=False)
        else:
            return sns.distplot(data, ax=ax)
    if data.shape[1] == 2:
        if data['x'].dtype.kind == 'O':
            sns.violinplot(x="x", y="y", data=data, ax=ax)
        else:
            #return sns.jointplot(x="x", y="y", data=data, kind="kde", ax=ax)
            return ax.hist2d(data['x'], data['y'], bins=100)
    else:
        return sns.distplot(data, ax=ax)

###  Evaluation metrics

In [ ]:
def filter_column_data_types(data_frame):
    categorical, continuous = [], []
    for name, dtype in data_frame.dtypes.items():
        if dtype.kind == "f":
            continuous.append(name)
        elif dtype.kind == "O":
            categorical.append(name)
    return categorical, continuous


def column_associations(data_frame):
    categorical, continuous = filter_column_data_types(data_frame)
    
    if len(continuous) > 0:
        continuous_df = data_frame[continuous]
        continuous_assoc = continuous_df.corr()
    else:
        continuous_assoc = None
    
    if len(categorical) > 0:
        categorical_df = data_frame[categorical]
        categorical_assoc = np.zeros(len(categorical), len(categorical))
        if len(continuous) > 0:
            mixed_assoc = np.zeros(len(categorical), len(continuous))
        for i in range(len(categorical)):
            col_i = categorical_df[categorical[i]]
            for j in range(i+1):
                col_j = categorical_df[categorical[j]]
                assoc  = 0.
                categorical_assoc[i, j] = assoc
                categorical_assoc[j, i] = assoc
            if len(continuous) > 0:
                for k in range(len(continuous)):
                    col_k = continuous_df[continuous[k]]
                    assoc = 0.
                    mixed_assoc[i, k] = assoc
    else:
        categorical_assoc, mixed_assoc = None, None
    return categorical_assoc, continuous_assoc, mixed_assoc


def max_correlation_distance(orig, synth):
    return np.abs((orig.corr() - synth.corr()).to_numpy()).max()

def mean_ks_distance(orig, synth):
    distances = [ks_2samp(orig[col], synth[col])[0] for col in orig.columns]
    return np.mean(distances)

In [ ]:
import time
from scipy.stats import ks_2samp
import matplotlib.pyplot as plt



default_metrics = {"avg_distance": mean_ks_distance}

def synthesize_and_plot(data, name, n_iter=evaluation.config['num_iterations'], metrics=None):
    start = time.time()
    with BasicSynthesizer(df=data, **evaluation.config['params']) as synthesizer:
        synthesizer.learn(df_train=data, num_iterations=n_iter)
        synthesized = synthesizer.synthesize(num_rows=len(data))
        print('took', time.time() - start, 's')
        for key, metric in metrics.items():
            evaluation[key] = metric(orig=data, synth=synthesized)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5), sharex=True, sharey=True)
        ax1.set_title('orig')
        ax2.set_title('synth')
        plot_data(data, ax=ax1)
        plot_data(synthesized, ax=ax2)

### Gauss "ball" outside of center

In [ ]:
data = create_ball(x_mean=1000, x_std=100, y_mean=100, y_std=10, size=10000)

In [ ]:
synthesize_and_plot(data, 'ball', metrics=default_metrics)

### Gauss "ball" around of zero

In [ ]:
data = create_ball(x_mean=0, x_std=100, y_mean=0, y_std=10, size=10000)

In [ ]:
synthesize_and_plot(data, 'ball_ext', metrics=default_metrics)

### Correlated Gaussian far from zero

In [ ]:
data = create_ball(x_mean=1000, x_std=100, y_mean=100, y_std=10, size=10000, cor=0.8, eps=1e-6)

In [ ]:
metrics = dict(default_metrics)
metrics["max_correlation_distance"] = max_correlation_distance
synthesize_and_plot(data, 'corr_ball_far', metrics=metrics)

### Correlated Gaussian around zero

In [ ]:
data = create_ball(x_mean=0, x_std=100, y_mean=0, y_std=10, size=10000, cor=0.8, eps=1e-6)

In [ ]:
metrics = dict(default_metrics)
metrics["max_correlation_distance"] = max_correlation_distance
synthesize_and_plot(data, 'corr_ball_zero', metrics=metrics)

### Line of noise that far from zero

In [ ]:
data = create_line(x_range=(0, 1000), intercept=100, slope=-0.1, y_std=10, size=10000)

In [ ]:
synthesize_and_plot(data, 'line', metrics=default_metrics)

### Line of noise that comes from zero

In [ ]:
data = create_line(x_range=(0, 1000), intercept=0, slope=0.1, y_std=10, size=10000)

In [ ]:
synthesize_and_plot(data, 'line_ext', metrics=default_metrics)

### Power law distribution

In [ ]:
data = create_power_law_distribution(a=0.5, scale=1000, size=10000)

In [ ]:
synthesize_and_plot(data, 'power_law', metrics=default_metrics)

### Conditional distribution

In [ ]:
data = create_conditional_distibution((10,2), (20, 5), (30, 1), size=10000)

In [ ]:
synthesize_and_plot(data, 'conditional', metrics=default_metrics)

### Bernoulli distribution

In [ ]:
data = create_bernoulli_distribution(ratio=0.5, size=10000)

In [ ]:
synthesize_and_plot(data, 'bernoulli_50/50', metrics=default_metrics)

In [ ]:
data = create_bernoulli_distribution(ratio=0.2, size=10000)

In [ ]:
synthesize_and_plot(data, 'bernoulli_20/80', metrics=default_metrics)

### Categorical distribution

In [ ]:
data = create_unifom_categorical(n_classes=10, size=10000)

In [ ]:
synthesize_and_plot(data, 'categorical_uniform', metrics=default_metrics)

In [ ]:
data = create_power_law_categorical(n_classes=10, size=10000)

In [ ]:
synthesize_and_plot(data, 'categorical_powerlaw', metrics=default_metrics)

In [ ]:
evaluation.write_metrics()